In [1]:
import itertools
from json import dumps
from pprint import pprint

import torch
from numpy import arange
from pynimbar import loading_animation

from util import DenoiseTransform, ColorSpaceTransform, NormalizeTransform, evaluate_model, get_model_data

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
def process_cs_combination(target_space, device):
	(
		colorspace_train_loader,
		colorspace_test_loader,
		colorspace_validation_loader,
	) = get_model_data([ColorSpaceTransform(source_space="RGB", target_space=target_space)])

	colorspace_precision = evaluate_model(
		device,
		colorspace_train_loader,
		colorspace_test_loader,
		colorspace_validation_loader,
		verbose=False,
	)

	# Free up memory
	del colorspace_train_loader
	del colorspace_test_loader
	del colorspace_validation_loader
	torch.cuda.empty_cache()

	return {
		"target_space": target_space,
		"precision": colorspace_precision,
	}

In [4]:
def process_d_combination(template_window_size, search_window_size, device):
	(
		denoise_train_loader,
		denoise_test_loader,
		denoise_validation_loader,
	) = get_model_data([DenoiseTransform(template_window_size, search_window_size)])

	denoise_precision = evaluate_model(
		device,
		denoise_train_loader,
		denoise_test_loader,
		denoise_validation_loader,
		verbose=False,
	)

	# Free up memory
	del denoise_train_loader
	del denoise_test_loader
	del denoise_validation_loader
	torch.cuda.empty_cache()

	return {
		"template_window_size": template_window_size,
		"search_window_size": search_window_size,
		"precision": denoise_precision,
	}

In [5]:
def process_n_combination(mean, std, device):
	(
		denoise_train_loader,
		denoise_test_loader,
		denoise_validation_loader,
	) = get_model_data([NormalizeTransform(mean, std)])

	denoise_precision = evaluate_model(
		device,
		denoise_train_loader,
		denoise_test_loader,
		denoise_validation_loader,
		verbose=False,
	)

	# Free up memory
	del denoise_train_loader
	del denoise_test_loader
	del denoise_validation_loader
	torch.cuda.empty_cache()

	return {
		"mean": mean,
		"std": std,
		"precision": denoise_precision,
	}

In [ ]:
results = []

target_spaces = ["BGR", "HSV", "LAB", "YUV"]

for target_space in target_spaces:
	with loading_animation(
		f"Running with target_space: {target_space}...",
		break_on_error=True,
		verbose_errors=True,
		time_it_live=True,
	):
		result = process_cs_combination(target_space, device)
		results.append(result)

results.sort(key=lambda x: x["precision"][0], reverse=True)
pprint(results)

with open("./params/colorspace_high_con.json", "w+") as f:
	f.write(dumps(results))

In [6]:
results = []

combinations = list(itertools.product(range(5, 11), range(20, 26)))

for t, s in combinations:
	with loading_animation(
		f"Running with template_window_size: {t}, search_window_size: {s}...",
		break_on_error=True,
		verbose_errors=True,
		time_it_live=True,
	):
		result = process_d_combination(t, s, device)
		results.append(result)

results.sort(key=lambda x: x["precision"][0], reverse=True)
pprint(results)

with open("./params/denoise_high_con.json", "w+") as f:
	f.write(dumps(results))

Running with template_window_size: 5, search_window_size: 20... ok (elapsed time: 274.01 seconds)
Running with template_window_size: 5, search_window_size: 21... ok (elapsed time: 270.17 seconds)
Running with template_window_size: 5, search_window_size: 22... ok (elapsed time: 293.65 seconds)
Running with template_window_size: 5, search_window_size: 23... ok (elapsed time: 294.88 seconds)
Running with template_window_size: 5, search_window_size: 24... ok (elapsed time: 327.10 seconds)
Running with template_window_size: 5, search_window_size: 25... ok (elapsed time: 332.39 seconds)
Running with template_window_size: 6, search_window_size: 20... ok (elapsed time: 275.82 seconds)
Running with template_window_size: 6, search_window_size: 21... ok (elapsed time: 273.17 seconds)
Running with template_window_size: 6, search_window_size: 22... ok (elapsed time: 302.69 seconds)
Running with template_window_size: 6, search_window_size: 23... ok (elapsed time: 301.25 seconds)
Running with templat

In [ ]:
results = []

combinations = list(itertools.product(arange(0.15, 1, 0.15), arange(0.15, 1, 0.15)))

for mean, std in combinations:
	mean = round(mean, 2)
	std = round(std, 2)

	with loading_animation(
		f"Running with mean: {mean}, std: {std}...",
		break_on_error=True,
		verbose_errors=True,
		time_it_live=True,
	):
		result = process_n_combination(mean, std, device)
		results.append(result)

results.sort(key=lambda x: x["precision"][0], reverse=True)
pprint(results)

with open("./params/normalize_high_con.json", "w+") as f:
	f.write(dumps(results))

Running with mean: 0.15, std: 0.15... ok (elapsed time: 26.50 seconds)
Running with mean: 0.15, std: 0.3... ok (elapsed time: 24.29 seconds)
Running with mean: 0.15, std: 0.45... ok (elapsed time: 24.20 seconds)
Running with mean: 0.15, std: 0.6... ok (elapsed time: 23.99 seconds)
Running with mean: 0.15, std: 0.75... ok (elapsed time: 23.99 seconds)
Running with mean: 0.15, std: 0.9... ok (elapsed time: 24.63 seconds)
Running with mean: 0.3, std: 0.15... ok (elapsed time: 23.98 seconds)
Running with mean: 0.3, std: 0.3... ok (elapsed time: 23.99 seconds)
Running with mean: 0.3, std: 0.45... ok (elapsed time: 24.09 seconds)
Running with mean: 0.3, std: 0.6... ok (elapsed time: 24.10 seconds)
Running with mean: 0.3, std: 0.75... ok (elapsed time: 24.08 seconds)
Running with mean: 0.3, std: 0.9... ok (elapsed time: 24.09 seconds)
Running with mean: 0.45, std: 0.15... ok (elapsed time: 23.99 seconds)
Running with mean: 0.45, std: 0.3... ok (elapsed time: 25.41 seconds)
Running with mean: 